In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import pycaret
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

C:\Users\tjsim\Anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\tjsim\Anaconda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\tjsim\Anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [9]:
nba_odds_no_covid_07_22 = pd.concat(
    map(pd.read_csv,['../nss_capstone/df_21_22.csv',
                     #'../nss_capstone/df_20_21_covid.csv',
                     '../nss_capstone/df_19_20_covid.csv',
                     '../nss_capstone/df_18_19.csv',
                     '../nss_capstone/df_17_18.csv',
                     '../nss_capstone/df_16_17.csv',
                     '../nss_capstone/df_15_16.csv',
                     '../nss_capstone/df_14_15.csv',
                     '../nss_capstone/df_13_14.csv',
                     '../nss_capstone/df_12_13.csv',
                     '../nss_capstone/df_11_12_lockout.csv',
                     '../nss_capstone/df_10_11.csv',
                     '../nss_capstone/df_09_10.csv',
                     '../nss_capstone/df_08_09.csv',
                     '../nss_capstone/df_07_08.csv']), ignore_index = True).fillna('')  

In [3]:
#nba_odds_no_covid_07_22.to_csv('nba_odds_no_covid_07_22.csv') 

In [ ]:
df_train = pd.read_csv('../nss_capstone/nba_odds_train.csv').drop(columns =  ['Unnamed: 0.1','Unnamed: 0']).fillna(0)

****************GRADIENT BOOST*****************

In [18]:
#(35974/2) train data is comprised of 17,987 games 

In [76]:
df_train = pd.read_csv('../nss_capstone/nba_odds_no_covid_07_22.csv').drop(columns =  ['Unnamed: 0.1','Unnamed: 0']).fillna(0)

df_train['W/L_bool'] = ''

for i in range(0, len(df_train), 2):
    if df_train.iloc[i]['Final'] > df_train.iloc[i+1]['Final']:
        df_train.at[i, 'W/L_bool'] = '1'
        df_train.at[i+1, 'W/L_bool'] = '0'
    else:
        df_train.at[i, 'W/L_bool'] = '0'
        df_train.at[i+1, 'W/L_bool'] = '1'

variables = ['ML',
             'Open Total',
             'Close Total',
            'Open Spread',
            'Close Spread']

X = df_train[variables]

y = df_train['W/L_bool']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)

gbr = GradientBoostingClassifier(n_estimators = 3000).fit(X_train, y_train)



print(classification_report(y_test, gbr.predict(X_test)))

accuracy_score(y_test, gbr.predict(X_test))

# top left = true negative
# top right = false positive
# bottom left = false negative
# bottom right = true positive

confusion_matrix(y_test, gbr.predict(X_test))

              precision    recall  f1-score   support

           0       0.69      0.65      0.67      4497
           1       0.67      0.71      0.69      4497

    accuracy                           0.68      8994
   macro avg       0.68      0.68      0.68      8994
weighted avg       0.68      0.68      0.68      8994



array([[2938, 1559],
       [1319, 3178]], dtype=int64)

In [77]:
df_test = pd.read_csv('../nss_capstone/df_22_23.csv').drop(columns =  'Unnamed: 0').fillna(0)


df_test['W/L_bool'] = ''


for i in range(0, len(df_test), 2):
    if df_test.iloc[i]['Final'] > df_test.iloc[i+1]['Final']:
        df_test.at[i, 'W/L_bool'] = '1'
        df_test.at[i+1, 'W/L_bool'] = '0'
    else:
        df_test.at[i, 'W/L_bool'] = '0'
        df_test.at[i+1, 'W/L_bool'] = '1'
        
        

X_test = df_test[variables]
y_test = df_test['W/L_bool']

y_pred_test = gbr.predict(X_test)


print(classification_report(y_test, y_pred_test))

#confusion_matrix(y_test, gbr.predict(y_test))

              precision    recall  f1-score   support

           0       0.63      0.56      0.59       664
           1       0.60      0.67      0.63       664

    accuracy                           0.61      1328
   macro avg       0.62      0.61      0.61      1328
weighted avg       0.62      0.61      0.61      1328



*******************XGBOOST***************

In [78]:
df_xg_train = pd.read_csv('../nss_capstone/nba_odds_no_covid_07_22.csv').drop(columns =  ['Unnamed: 0.1','Unnamed: 0']).fillna(0)

df_xg_train['W/L_bool'] = ''

for i in range(0, len(df_xg_train), 2):
    if df_xg_train.iloc[i]['Final'] > df_xg_train.iloc[i+1]['Final']:
        df_xg_train.at[i, 'W/L_bool'] = '1'
        df_xg_train.at[i+1, 'W/L_bool'] = '0'
    else:
        df_xg_train.at[i, 'W/L_bool'] = '0'
        df_xg_train.at[i+1, 'W/L_bool'] = '1'

variables = ['ML',
             'Open Total',
             'Close Total',
            'Open Spread',
            'Close Spread']

X = df_xg_train[variables]

y = df_xg_train['W/L_bool'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)

xgboost_model = xgb.XGBClassifier(n_estimators=2000)
xgboost_model.fit(X_train, y_train)

y_pred = xgboost_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.59      0.63      4497
           1       0.64      0.72      0.68      4497

    accuracy                           0.66      8994
   macro avg       0.66      0.66      0.65      8994
weighted avg       0.66      0.66      0.65      8994



In [79]:
df_xg_test = pd.read_csv('../nss_capstone/df_22_23.csv').drop(columns =  'Unnamed: 0').fillna(0)

df_xg_test['W/L_bool'] = ''

for i in range(0, len(df_xg_test), 2):
    if df_xg_test.iloc[i]['Final'] > df_xg_test.iloc[i+1]['Final']:
        df_xg_test.at[i, 'W/L_bool'] = '1'
        df_xg_test.at[i+1, 'W/L_bool'] = '0'
    else:
        df_xg_test.at[i, 'W/L_bool'] = '0'
        df_xg_test.at[i+1, 'W/L_bool'] = '1'

X_test = df_xg_test[variables]
y_test = df_xg_test['W/L_bool'].astype(int)

y_pred = xgboost_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.54      0.56       664
           1       0.57      0.62      0.60       664

    accuracy                           0.58      1328
   macro avg       0.58      0.58      0.58      1328
weighted avg       0.58      0.58      0.58      1328



**********************NAIVE BAYES*************************

In [86]:
df_train = pd.read_csv('../nss_capstone/nba_odds_no_covid_07_22.csv').drop(columns =  ['Unnamed: 0.1','Unnamed: 0']).fillna(0)

df_train['W/L_bool'] = ''

for i in range(0, len(df_train), 2):
    if df_train.iloc[i]['Final'] > df_train.iloc[i+1]['Final']:
        df_train.at[i, 'W/L_bool'] = '1'
        df_train.at[i+1, 'W/L_bool'] = '0'
    else:
        df_train.at[i, 'W/L_bool'] = '0'
        df_train.at[i+1, 'W/L_bool'] = '1'

variables = ['ML',
             'Open Total',
             'Close Total',
            'Open Spread',
            'Close Spread']

X = df_train[variables]

y = df_train['W/L_bool']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)


naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)

y_nb_pred = naive_bayes.predict(X_test)

print(classification_report(y_test, y_nb_pred))

# # top left = true negative
# # top right = false positive
# # bottom left = false negative
# # bottom right = true positive

confusion_matrix(y_test, naive_bayes.predict(X_test))

              precision    recall  f1-score   support

           0       0.67      0.74      0.70      4497
           1       0.71      0.63      0.67      4497

    accuracy                           0.69      8994
   macro avg       0.69      0.69      0.68      8994
weighted avg       0.69      0.69      0.68      8994



array([[3332, 1165],
       [1662, 2835]], dtype=int64)

In [87]:
df_nb_test = pd.read_csv('../nss_capstone/df_22_23.csv').drop(columns =  'Unnamed: 0').fillna(0)


df_nb_test['W/L_bool'] = ''

for i in range(0, len(df_nb_test), 2):
    if df_nb_test.iloc[i]['Final'] > df_nb_test.iloc[i+1]['Final']:
        df_nb_test.at[i, 'W/L_bool'] = '1'
        df_nb_test.at[i+1, 'W/L_bool'] = '0'
    else:
        df_nb_test.at[i, 'W/L_bool'] = '0'
        df_nb_test.at[i+1, 'W/L_bool'] = '1'
        

X_nb_test = df_nb_test[variables]
y_nb_test = df_nb_test['W/L_bool']#.astype(int)


y_pred_nb_test = naive_bayes.predict(X_nb_test)

print(classification_report(y_nb_test, y_pred_nb_test))

              precision    recall  f1-score   support

           0       0.62      0.71      0.66       664
           1       0.66      0.57      0.61       664

    accuracy                           0.64      1328
   macro avg       0.64      0.64      0.64      1328
weighted avg       0.64      0.64      0.64      1328



In [ ]:
RANDOM FOREST

In [88]:
df_rf_train = pd.read_csv('../nss_capstone/nba_odds_no_covid_07_22.csv').drop(columns =  ['Unnamed: 0.1','Unnamed: 0']).fillna(0)


df_rf_train['W/L_bool'] = ''


for i in range(0, len(df_rf_train), 2):
    if df_rf_train.iloc[i]['Final'] > df_rf_train.iloc[i+1]['Final']:
        df_rf_train.at[i, 'W/L_bool'] = '1'
        df_rf_train.at[i+1, 'W/L_bool'] = '0'
    else:
        df_rf_train.at[i, 'W/L_bool'] = '0'
        df_rf_train.at[i+1, 'W/L_bool'] = '1'


variables = ['ML', 'Open Total', 'Close Total', 'Open Spread', 'Close Spread']


X = df_rf_train[variables]
y = df_rf_train['W/L_bool'].astype(int)

# Split the data into training and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=321, stratify=y)

# Train the Random Forest Classifier

random_forest = RandomForestClassifier(n_estimators=3000)
random_forest.fit(X_train, y_train)

# Make predictions on the test set

y_pred = random_forest.predict(X_test)

# Evaluate the model's performance

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.62      0.65      3598
           1       0.65      0.70      0.67      3597

    accuracy                           0.66      7195
   macro avg       0.66      0.66      0.66      7195
weighted avg       0.66      0.66      0.66      7195



In [89]:
df_rf_test = pd.read_csv('../nss_capstone/df_22_23.csv').drop(columns =  'Unnamed: 0').fillna(0)

df_rf_test['W/L_bool'] = ''

for i in range(0, len(df_rf_test), 2):
    if df_rf_test.iloc[i]['Final'] > df_rf_test.iloc[i+1]['Final']:
        df_rf_test.at[i, 'W/L_bool'] = '1'
        df_rf_test.at[i+1, 'W/L_bool'] = '0'
    else:
        df_rf_test.at[i, 'W/L_bool'] = '0'
        df_rf_test.at[i+1, 'W/L_bool'] = '1'

X_test = df_rf_test[variables]
y_test = df_rf_test['W/L_bool'].astype(int)

y_pred_test = random_forest.predict(X_test)

print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.58      0.56      0.57       664
           1       0.57      0.60      0.58       664

    accuracy                           0.58      1328
   macro avg       0.58      0.58      0.58      1328
weighted avg       0.58      0.58      0.58      1328



***************************1st_half/ATS for fav, based off score diff*******************************************

In [260]:
df = pd.read_csv('../nss_capstone/nba_odds_no_covid_07_22.csv').drop(columns =  ['Unnamed: 0.1','Unnamed: 0']).fillna(0)

In [261]:
df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Open Total,Close Total,Open Spread,Close Spread,W/L
0,2022-06-16,528,H,Boston,22,17,27,24,90,2.5,4.0,-175.0,5.5,0.0,0.0,2.5,4.0,L
1,2022-06-16,527,V,Golden State,27,27,22,27,103,212.5,211.5,155.0,103.5,212.5,211.5,0.0,0.0,W
2,2022-06-13,526,H,Golden State,27,24,24,29,104,3.5,4.0,-165.0,107.5,0.0,0.0,3.5,4.0,W
3,2022-06-13,525,V,Boston,16,23,35,20,94,212.5,211.0,145.0,2.5,212.5,211.0,0.0,0.0,L
4,2022-06-10,524,H,Boston,28,26,24,19,97,2.5,4.0,-165.0,1,0.0,0.0,2.5,4.0,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35969,2007-10-30,505,V,Houston,16,27,27,25,95,2.5,5.0,-230.0,3,0.0,0.0,2.5,5.0,W
35970,2007-10-30,504,H,GoldenState,30,21,21,24,96,3.0,1.0,-120.0,3,0.0,0.0,3.0,1.0,L
35971,2007-10-30,503,V,Utah,28,34,24,31,117,214.5,212.0,100.0,105.5,214.5,212.0,0.0,0.0,W
35972,2007-10-30,502,H,SanAntonio,29,30,22,25,106,12.5,13.0,-1400.0,5,0.0,0.0,12.5,13.0,W


In [262]:
df['W/L_bool'] = ''

for i in range(0, len(df), 2):
    if df.iloc[i]['Final'] > df.iloc[i+1]['Final']:
        df.at[i, 'W/L_bool'] = '1'
        df.at[i+1, 'W/L_bool'] = '0'
    else:
        df.at[i, 'W/L_bool'] = '0'
        df.at[i+1, 'W/L_bool'] = '1'

In [263]:
df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Open Total,Close Total,Open Spread,Close Spread,W/L,W/L_bool
0,2022-06-16,528,H,Boston,22,17,27,24,90,2.5,4.0,-175.0,5.5,0.0,0.0,2.5,4.0,L,0
1,2022-06-16,527,V,Golden State,27,27,22,27,103,212.5,211.5,155.0,103.5,212.5,211.5,0.0,0.0,W,1
2,2022-06-13,526,H,Golden State,27,24,24,29,104,3.5,4.0,-165.0,107.5,0.0,0.0,3.5,4.0,W,1
3,2022-06-13,525,V,Boston,16,23,35,20,94,212.5,211.0,145.0,2.5,212.5,211.0,0.0,0.0,L,0
4,2022-06-10,524,H,Boston,28,26,24,19,97,2.5,4.0,-165.0,1,0.0,0.0,2.5,4.0,L,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35969,2007-10-30,505,V,Houston,16,27,27,25,95,2.5,5.0,-230.0,3,0.0,0.0,2.5,5.0,W,1
35970,2007-10-30,504,H,GoldenState,30,21,21,24,96,3.0,1.0,-120.0,3,0.0,0.0,3.0,1.0,L,0
35971,2007-10-30,503,V,Utah,28,34,24,31,117,214.5,212.0,100.0,105.5,214.5,212.0,0.0,0.0,W,1
35972,2007-10-30,502,H,SanAntonio,29,30,22,25,106,12.5,13.0,-1400.0,5,0.0,0.0,12.5,13.0,W,1


In [264]:
df['first_half'] = (df['1st'] + df['2nd']).groupby(df.index // 2).transform(lambda x: (x == x.max()).astype(int))

In [265]:
df['Score_Diff'] = np.where(
    df.groupby(df.index // 2)['Close Spread'].transform('first') > 0,
    df.groupby(df.index // 2)['Final'].diff(-1),
    df.groupby(df.index // 2)['Final'].diff(1)
)


In [266]:
df['ATS'] = np.where(df['Score_Diff'] > 0, 1, 0)
df['ATS'] = df['ATS'].fillna(0).astype(int)

In [267]:
#df = pd.read_csv('../nss_capstone/nba_odds_no_covid_07_22.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0']).fillna(0)

df['first_half'] = (df['1st'] + df['2nd']).groupby(df.index // 2).transform(lambda x: (x == x.max()).astype(int))

df['Score_Diff'] = np.where(
    df.groupby(df.index // 2)['Close Spread'].transform('first') > 0,
    df.groupby(df.index // 2)['Final'].diff(-1),
    df.groupby(df.index // 2)['Final'].diff(1)
)

df['ATS'] = np.where(df['Score_Diff'] > df['Close Spread'], 1, 0)
df['ATS'] = df['ATS'].fillna(0).astype(int)

In [268]:
df.columns

Index(['Date', 'Rot', 'VH', 'Team', '1st', '2nd', '3rd', '4th', 'Final',
       'Open', 'Close', 'ML', '2H', 'Open Total', 'Close Total', 'Open Spread',
       'Close Spread', 'W/L', 'W/L_bool', 'first_half', 'Score_Diff', 'ATS'],
      dtype='object')

In [270]:
df[['Team', 'Final', 'Close Spread','Score_Diff', 'ATS', 'W/L_bool']].head(10)

,Team,Final,Close Spread,Score_Diff,ATS,W/L_bool
0,Boston,90,4.0,-13.0,0,0
1,Golden State,103,0.0,NaN,0,1
2,Golden State,104,4.0,10.0,1,1
3,Boston,94,0.0,NaN,0,0
4,Boston,97,4.0,-10.0,0,0
5,GoldenState,107,0.0,NaN,0,1
6,Boston,116,3.5,16.0,1,1
7,GoldenState,100,0.0,NaN,0,0
8,GoldenState,107,5.0,19.0,1,1
9,Boston,88,0.0,NaN,0,0


In [272]:
df.to_csv('../nss_capstone/df.csv')

In [276]:
df = pd.read_csv('../nss_capstone/df.csv').drop(columns = 'Unnamed: 0').fillna('')
df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,...,2H,Open Total,Close Total,Open Spread,Close Spread,W/L,W/L_bool,first_half,Score_Diff,ATS
0,2022-06-16,528,H,Boston,22,17,27,24,90,2.5,...,5.5,0.0,0.0,2.5,4.0,L,0,0,-13.0,0
1,2022-06-16,527,V,Golden State,27,27,22,27,103,212.5,...,103.5,212.5,211.5,0.0,0.0,W,1,1,,0
2,2022-06-13,526,H,Golden State,27,24,24,29,104,3.5,...,107.5,0.0,0.0,3.5,4.0,W,1,1,10.0,1
3,2022-06-13,525,V,Boston,16,23,35,20,94,212.5,...,2.5,212.5,211.0,0.0,0.0,L,0,0,,0
4,2022-06-10,524,H,Boston,28,26,24,19,97,2.5,...,1,0.0,0.0,2.5,4.0,L,0,1,-10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35969,2007-10-30,505,V,Houston,16,27,27,25,95,2.5,...,3,0.0,0.0,2.5,5.0,W,1,1,2.0,0
35970,2007-10-30,504,H,GoldenState,30,21,21,24,96,3.0,...,3,0.0,0.0,3.0,1.0,L,0,0,-21.0,0
35971,2007-10-30,503,V,Utah,28,34,24,31,117,214.5,...,105.5,214.5,212.0,0.0,0.0,W,1,1,,0
35972,2007-10-30,502,H,SanAntonio,29,30,22,25,106,12.5,...,5,0.0,0.0,12.5,13.0,W,1,1,9.0,0


In [277]:

variables = ['ML',
             'first_half',
             'Open Total',
             'Close Total',
            'Open Spread',
            'Close Spread']

X = df[variables]

y = df['W/L_bool']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)

gbr = GradientBoostingClassifier(n_estimators = 1000).fit(X_train, y_train)



print(classification_report(y_test, gbr.predict(X_test)))

accuracy_score(y_test, gbr.predict(X_test))

# top left = true negative
# top right = false positive
# bottom left = false negative
# bottom right = true positive

confusion_matrix(y_test, gbr.predict(X_test))

              precision    recall  f1-score   support

           0       0.75      0.73      0.74      4497
           1       0.74      0.75      0.75      4497

    accuracy                           0.74      8994
   macro avg       0.74      0.74      0.74      8994
weighted avg       0.74      0.74      0.74      8994



array([[3305, 1192],
       [1115, 3382]], dtype=int64)

In [312]:

variables = ['ML',
             'ATS',
             'first_half',
             #'Open Total',
             #'Close Total',
            #'Open Spread',
            'Close Spread']

X = df[variables]

y = df['W/L_bool']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)

gbr = GradientBoostingClassifier(n_estimators = 1000).fit(X_train, y_train)



print(classification_report(y_test, gbr.predict(X_test)))

accuracy_score(y_test, gbr.predict(X_test))

# top left = true negative
# top right = false positive
# bottom left = false negative
# bottom right = true positive

confusion_matrix(y_test, gbr.predict(X_test))

              precision    recall  f1-score   support

           0       0.79      0.84      0.81      4497
           1       0.83      0.78      0.80      4497

    accuracy                           0.81      8994
   macro avg       0.81      0.81      0.81      8994
weighted avg       0.81      0.81      0.81      8994



array([[3777,  720],
       [1011, 3486]], dtype=int64)

Clean df and predict year

In [279]:
df_22_23 = pd.read_csv('nba_odds_22_23.csv')#.drop(columns =  'Unnamed: 0').fillna(0)
#df_22_23 = df_22_23.iloc[::-1]

In [280]:
df_22_23

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,1018,501,V,Philadelphia,29,34,25,29,117,229,216.0,135,107
1,1018,502,H,Boston,24,39,35,28,126,7,3.0,-155,2
2,1018,503,V,LALakers,22,30,19,38,109,229.5,223.5,260,114
3,1018,504,H,GoldenState,25,34,32,32,123,6.5,7.5,-310,2
4,1019,505,V,Washington,36,24,27,27,114,2,2.5,-135,111.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,116,514,H,Minnesota,27,31,37,30,125,0.5,4.0,-175,pk
1324,116,515,V,Phoenix,36,29,23,18,106,225.5,231.0,550,118
1325,116,516,H,Memphis,29,39,39,29,136,10,12.5,-800,6.5
1326,116,517,V,Houston,35,26,37,34,132,234.5,236.5,240,116


In [281]:
df_22_23  = df_22_23.replace({'Open': 'pk', 'Close': 'pk', '2H': 'pk'}, 0)

In [282]:
df_22_23['Open'] = pd.to_numeric(df_22_23['Open'], errors='coerce')
df_22_23['Close'] = pd.to_numeric(df_22_23['Close'], errors='coerce')

df_22_23.loc[df_22_23['Open'] >= 100, 'Open Total'] = df_22_23['Open']
df_22_23.loc[df_22_23['Close'] >= 100, 'Close Total'] = df_22_23['Close']

df_22_23.loc[df_22_23['Open'] <= 30, 'Open Spread'] = df_22_23['Open']
df_22_23.loc[df_22_23['Close'] <= 30, 'Close Spread'] = df_22_23['Close']

In [283]:
df_22_23['W/L'] = ''

# Loop over the rows in pairs and compare the Final values
for i in range(0, len(df_22_23), 2):
    if df_22_23.iloc[i]['Final'] > df_22_23.iloc[i+1]['Final']:
        df_22_23.at[i, 'W/L'] = 'W'
        df_22_23.at[i+1, 'W/L'] = 'L'
    else:
        df_22_23.at[i, 'W/L'] = 'L'
        df_22_23.at[i+1, 'W/L'] = 'W'

In [284]:
df_22_23

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Open Total,Close Total,Open Spread,Close Spread,W/L
0,1018,501,V,Philadelphia,29,34,25,29,117,229.0,216.0,135,107,229.0,216.0,NaN,NaN,L
1,1018,502,H,Boston,24,39,35,28,126,7.0,3.0,-155,2,NaN,NaN,7.0,3.0,W
2,1018,503,V,LALakers,22,30,19,38,109,229.5,223.5,260,114,229.5,223.5,NaN,NaN,L
3,1018,504,H,GoldenState,25,34,32,32,123,6.5,7.5,-310,2,NaN,NaN,6.5,7.5,W
4,1019,505,V,Washington,36,24,27,27,114,2.0,2.5,-135,111.5,NaN,NaN,2.0,2.5,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,116,514,H,Minnesota,27,31,37,30,125,0.5,4.0,-175,0,NaN,NaN,0.5,4.0,L
1324,116,515,V,Phoenix,36,29,23,18,106,225.5,231.0,550,118,225.5,231.0,NaN,NaN,L
1325,116,516,H,Memphis,29,39,39,29,136,10.0,12.5,-800,6.5,NaN,NaN,10.0,12.5,W
1326,116,517,V,Houston,35,26,37,34,132,234.5,236.5,240,116,234.5,236.5,NaN,NaN,L


In [285]:
df_22_23[['Team', 'Final', 'W/L']].head(2)

,Team,Final,W/L
0,Philadelphia,117,L
1,Boston,126,W


In [286]:
df_22_23['W/L_bool'] = ''

for i in range(0, len(df_22_23), 2):
    if df_22_23.iloc[i]['Final'] > df_22_23.iloc[i+1]['Final']:
        df_22_23.at[i, 'W/L_bool'] = '1'
        df_22_23.at[i+1, 'W/L_bool'] = '0'
    else:
        df_22_23.at[i, 'W/L_bool'] = '0'
        df_22_23.at[i+1, 'W/L_bool'] = '1'

In [287]:
df_22_23

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Open Total,Close Total,Open Spread,Close Spread,W/L,W/L_bool
0,1018,501,V,Philadelphia,29,34,25,29,117,229.0,216.0,135,107,229.0,216.0,NaN,NaN,L,0
1,1018,502,H,Boston,24,39,35,28,126,7.0,3.0,-155,2,NaN,NaN,7.0,3.0,W,1
2,1018,503,V,LALakers,22,30,19,38,109,229.5,223.5,260,114,229.5,223.5,NaN,NaN,L,0
3,1018,504,H,GoldenState,25,34,32,32,123,6.5,7.5,-310,2,NaN,NaN,6.5,7.5,W,1
4,1019,505,V,Washington,36,24,27,27,114,2.0,2.5,-135,111.5,NaN,NaN,2.0,2.5,W,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,116,514,H,Minnesota,27,31,37,30,125,0.5,4.0,-175,0,NaN,NaN,0.5,4.0,L,0
1324,116,515,V,Phoenix,36,29,23,18,106,225.5,231.0,550,118,225.5,231.0,NaN,NaN,L,0
1325,116,516,H,Memphis,29,39,39,29,136,10.0,12.5,-800,6.5,NaN,NaN,10.0,12.5,W,1
1326,116,517,V,Houston,35,26,37,34,132,234.5,236.5,240,116,234.5,236.5,NaN,NaN,L,0


In [288]:
df_22_23['first_half'] = (df_22_23['1st'] + df_22_23['2nd']).groupby(df_test.index // 2).transform(lambda x: (x == x.max()).astype(int))

In [289]:
df_22_23['Score_Diff'] = np.where(
    df_22_23.groupby(df_test.index // 2)['Close Spread'].transform('first') > 0,
    df_22_23.groupby(df_test.index // 2)['Final'].diff(-1),
    df_22_23.groupby(df_test.index // 2)['Final'].diff(1)
)


In [290]:
df_22_23

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,...,ML,2H,Open Total,Close Total,Open Spread,Close Spread,W/L,W/L_bool,first_half,Score_Diff
0,1018,501,V,Philadelphia,29,34,25,29,117,229.0,...,135,107,229.0,216.0,NaN,NaN,L,0,1,-9.0
1,1018,502,H,Boston,24,39,35,28,126,7.0,...,-155,2,NaN,NaN,7.0,3.0,W,1,1,NaN
2,1018,503,V,LALakers,22,30,19,38,109,229.5,...,260,114,229.5,223.5,NaN,NaN,L,0,0,-14.0
3,1018,504,H,GoldenState,25,34,32,32,123,6.5,...,-310,2,NaN,NaN,6.5,7.5,W,1,1,NaN
4,1019,505,V,Washington,36,24,27,27,114,2.0,...,-135,111.5,NaN,NaN,2.0,2.5,W,1,1,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,116,514,H,Minnesota,27,31,37,30,125,0.5,...,-175,0,NaN,NaN,0.5,4.0,L,0,1,NaN
1324,116,515,V,Phoenix,36,29,23,18,106,225.5,...,550,118,225.5,231.0,NaN,NaN,L,0,0,-30.0
1325,116,516,H,Memphis,29,39,39,29,136,10.0,...,-800,6.5,NaN,NaN,10.0,12.5,W,1,1,NaN
1326,116,517,V,Houston,35,26,37,34,132,234.5,...,240,116,234.5,236.5,NaN,NaN,L,0,0,-8.0


In [291]:
df_22_23['ATS'] = np.where(df_22_23['Score_Diff'] > df_22_23['Close Spread'], 1, 0)
df_22_23['ATS'] = df_22_23['ATS'].fillna(0).astype(int)

In [303]:
df_22_23[['Date','Team', 'Final', 'Close Spread','Score_Diff','W/L_bool', 'ATS']].head(4)

,Date,Team,Final,Close Spread,Score_Diff,W/L_bool,ATS
0,1018,Philadelphia,117,NaN,-9.0,0,0
1,1018,Boston,126,3.0,NaN,1,0
2,1018,LALakers,109,NaN,-14.0,0,0
3,1018,GoldenState,123,7.5,NaN,1,0


In [293]:
df_22_23.to_csv('../nss_capstone/df_22_23.csv')

In [300]:
df_val = pd.read_csv('../nss_capstone/df_22_23.csv').drop(columns = 'Unnamed: 0').fillna(0)

In [301]:
df_val

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,...,2H,Open Total,Close Total,Open Spread,Close Spread,W/L,W/L_bool,first_half,Score_Diff,ATS
0,1018,501,V,Philadelphia,29,34,25,29,117,229.0,...,107,229.0,216.0,0.0,0.0,L,0,1,-9.0,0
1,1018,502,H,Boston,24,39,35,28,126,7.0,...,2,0.0,0.0,7.0,3.0,W,1,1,0.0,0
2,1018,503,V,LALakers,22,30,19,38,109,229.5,...,114,229.5,223.5,0.0,0.0,L,0,0,-14.0,0
3,1018,504,H,GoldenState,25,34,32,32,123,6.5,...,2,0.0,0.0,6.5,7.5,W,1,1,0.0,0
4,1019,505,V,Washington,36,24,27,27,114,2.0,...,111.5,0.0,0.0,2.0,2.5,W,1,1,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,116,514,H,Minnesota,27,31,37,30,125,0.5,...,0,0.0,0.0,0.5,4.0,L,0,1,0.0,0
1324,116,515,V,Phoenix,36,29,23,18,106,225.5,...,118,225.5,231.0,0.0,0.0,L,0,0,-30.0,0
1325,116,516,H,Memphis,29,39,39,29,136,10.0,...,6.5,0.0,0.0,10.0,12.5,W,1,1,0.0,0
1326,116,517,V,Houston,35,26,37,34,132,234.5,...,116,234.5,236.5,0.0,0.0,L,0,0,-8.0,0


In [313]:
X_test = df_val[variables]
y_test = df_val['W/L_bool']#.astype(int)

y_pred_test = gbr.predict(X_test)

print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.66      0.84      0.74       664
           1       0.77      0.56      0.65       664

    accuracy                           0.70      1328
   macro avg       0.72      0.70      0.69      1328
weighted avg       0.72      0.70      0.69      1328

